# 2.1 More QC and filtering

**Inés Sentís**

Date of execution 

In [ ]:
Sys.Date()

## Introduction

Here we are going to perform some QC on the data itself. This report is following [Satija's Lab recomendations](https://satijalab.org/seurat/articles/pbmc3k_tutorial.html). The following QC metrics are well accepted QC cheks from single cell community. The final goal is to remove low-quality cells.

## Libraries

In [ ]:
suppressMessages(suppressWarnings({
library(Seurat)
library(ggpubr)
library(tidyverse)
library(here)
library(glue)
library(grid)
library(gridExtra)
library(reshape2)}))

## Parameters

In [ ]:
subproj <- "SCGRES_124_125"

In [ ]:
source(here(glue("{subproj}/sc_analysis/misc/paths.R")))
source(here("utils/bin.R"))

## Load data

In [ ]:
metadata <- read.csv(here(glue("{cellranger}/metadata.csv")))

In [ ]:
head(metadata)

In [ ]:
list_objects <- purrr::map(list.dirs(path = here(glue("{cellranger}/jobs")), full.names = FALSE, recursive = FALSE), function(lib){
    if (!grepl("test", lib)) {
        print(lib)
        info <- subset(metadata, type == "cDNA" & gem_id == lib)
        data <- Read10X(data.dir=here(glue("{cellranger}/jobs/{lib}/{lib}/outs/per_sample_outs/{lib}/count/sample_filtered_feature_bc_matrix")))
        seurat_obj <- CreateSeuratObject(counts=data, project=subproj)
        seurat_obj$sample_org <- lib
        seurat_obj$timepoint <- info$timepoint
        seurat_obj$replicate <- info$replicate
        seurat_obj$sample_id <- paste(info$timepoint,  info$replicate, sep="_")
        seurat_obj$sample_id <- info$gem_id
        seurat_obj
    }
})

In [ ]:
list_objects <- Filter(function(x) !is.null(x), list_objects)

In [ ]:
list_names <- lapply(list_objects, function(obj) {
    if (unique(obj$timepoint) != "T0") {
        paste(unique(obj$timepoint),unique(obj$replicate), sep='_')
    }else{
        name <- gsub("CC2269","", unique(obj$sample_org))
        paste(unique(obj$timepoint),name, sep='_')
    }
})
list_names

In [ ]:
## SCGRES_124_125

# list_names <- lapply(list_objects, function(obj) {
#     unique(obj$sample_id)
# })
# list_names

In [ ]:
list_objects <- setNames(list_objects, list_names)

## Apply straightforward actions

#### Remove empty genes

In [ ]:
list_objects <- lapply(list_objects, function(obj) {
  
  print(table(rowSums(as.matrix(obj[['RNA']]@counts)) == 0))
  
  keep_genes <- data.frame(rowSums(as.matrix(obj[['RNA']]@counts)) != 0)
  colnames(keep_genes) <- "logical"
  keep_genes <- subset(keep_genes, logical==TRUE)
  obj[['RNA']]@data <- obj[['RNA']]@data[rownames(keep_genes), ]
  return(obj)
})

In [ ]:
list_objects <- lapply(list_objects, function(obj) {
  # compute % of mitocondrial genes in cells
  obj[["pct_mt"]] <- PercentageFeatureSet(obj,pattern = "^MT-")
  
  # compute % of ribosomal genes in cells
  obj[["percent.ribo"]] <- PercentageFeatureSet(
  object = obj,
  pattern = "^RPL|^RPS")
  return(obj)
})

In [ ]:
list_objects <- setNames(list_objects, list_names)

## Define some thresholds according to distribution

In [ ]:
# ver <- "2022-04-1" 
# orientative filters
max_lib_size <- 80000
min_lib_size <- 1000 #if a sample has a good coverage (>=minCov), then don't set a lower thresold for nCount, it's already pretty good.
max_n_genes <- 50000
min_n_genes <- 200
max_pct_mt <- 20
max_pct_ribo <- 50

filters <- data.frame(sample=character(),
  min_lib_size=numeric(),
  max_lib_size=numeric(),
  min_n_genes=numeric(),
  max_n_genes = numeric())

for (i in seq_along(list_objects)) {
  print(i)
  id <-names(list_objects)[i]
  obj <- list_objects[[id]]

  if (min(obj$nCount_RNA)>=min_lib_size){
      min_lib_size=min(obj$nCount_RNA)
    }else{
      min_lib_size=quantile(obj$nCount_RNA, prob=c(0.05))  
  }
  max_lib_size=quantile(obj$nCount_RNA, prob=0.99) 
  min_n_genes=quantile(obj$nFeature_RNA, prob=0.10)
  max_n_genes=quantile(obj$nFeature_RNA, prob=0.99) 
  
  
  df <- data.frame(sample=character(),
  min_lib_size=numeric(),
  max_lib_size=numeric(),
  min_n_genes=numeric(),
  max_n_genes = numeric())
  
  df[nrow(df) + 1,] <- c(as.character(id), as.numeric(min_lib_size[[1]]), 
              as.numeric(max_lib_size[[1]]), as.numeric(min_n_genes[[1]]), 
              as.numeric(max_n_genes[[1]]))
  
  filters <- rbind(filters,df)
}

In [ ]:
# manually adjust them if need it
filters$min_lib_size <- 1000

## RNA assay sanity checks

### Library size

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 5, warn=-1,verbose = FALSE)

suppressMessages(lapply(seq_along(list_objects), function(i) {
  id <- names(list_objects[i])
  obj <- list_objects[[i]]
  
  df<- filters %>% dplyr::filter(sample == id) 
  df$max_lib_size = as.numeric(df$max_lib_size)
  df$min_lib_size = as.numeric(df$min_lib_size)
  df$min_n_genes = as.numeric(df$min_n_genes)
  df$max_n_genes = as.numeric(df$max_n_genes)
  
  vlp <- VlnPlot(
  object = obj,
  features = "nCount_RNA",
  pt.size = 0.1) +  
  scale_y_log10() + 
  geom_hline(yintercept = df$max_lib_size, linetype='dashed', col = 'black') +
  geom_hline(yintercept = df$min_lib_size, linetype = "dashed", color = "black") +
  theme(legend.position = 'none')
  
  hist <- plot_histogram_qc(df = obj@meta.data,
                          x = "nCount_RNA", 
                          x_lab = "Library Size (log10(total UMI))")
  hist1 <- hist +
      scale_x_log10() +
      geom_vline(xintercept = df$max_lib_size, 
                linetype = "dashed", color = "black") +
      geom_vline(xintercept = df$min_lib_size, 
                linetype = "dashed", color = "black")
  
  hist2 <- hist +
      scale_x_continuous(limits = c(0, 2000)) +
      xlab("Library Size (total UMI)") +
      theme_pubr() +
      geom_vline(xintercept = df$min_lib_size, linetype = "dashed", color = "black")
  
  grid.arrange(vlp, hist1, hist2, ncol=3, nrow =1,
               top = textGrob(id, gp=gpar(fontsize=20,font=3)))
}))


### Number of detected genes

In [ ]:
suppressMessages(lapply(seq_along(list_objects), function(i) {
  
  id <- names(list_objects[i])
  obj <- list_objects[[i]]
  
  df <- filters %>% dplyr::filter(sample == id) 
  df$max_lib_size = as.numeric(df$max_lib_size)
  df$min_lib_size = as.numeric(df$min_lib_size)
  df$min_n_genes = as.numeric(df$min_n_genes)
  df$max_n_genes = as.numeric(df$max_n_genes)
  
  vlp <- VlnPlot(
  object = obj,
  features = "nFeature_RNA",
  pt.size = 0.1) +  
  scale_y_log10() +
  geom_hline(yintercept = df$max_n_genes, linetype='dashed', col = 'black') +
  geom_hline(yintercept = df$min_n_genes, linetype = "dashed", color = "black") +
  theme(legend.position = 'none')
  
  hist1 <- plot_histogram_qc(df = obj@meta.data,
                            x = "nFeature_RNA", 
                            x_lab = "Number of Detected Genes") +
           geom_vline(xintercept = df$max_n_genes, linetype='dashed', col = 'black') +
           geom_vline(xintercept = df$min_n_genes, 
                linetype = "dashed", color = "black") 
  
  hist2 <- hist1 +
      scale_x_continuous(limits = c(0, 5000)) +
      geom_vline(xintercept = df$min_n_genes, linetype = "dashed", color = "black")
  
  grid.arrange(vlp, hist1, hist2, ncol=3, nrow =1,
               top = textGrob(id, gp=gpar(fontsize=20,font=3)))
}))

## Fraction of mitochondrial expression

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 10, warn=-1,verbose = FALSE)
list_plots <- lapply(seq_along(list_objects), function(i) {
  id <- names(list_objects[i])
  obj <- list_objects[[i]]
  plt <- obj@meta.data %>%
  plot_histogram_qc(x = "pct_mt", x_lab = "% Mitochondrial Expression") +
  geom_vline(xintercept = max_pct_mt, linetype = "dashed", color = "black") +
  scale_x_continuous(limits = c(0, 100)) +
  ggtitle(id) + 
  theme(plot.title = element_text(hjust = 0.5))
    
  return(plt)
})
cowplot::plot_grid(plotlist = list_plots,
                   align = "hv",
                   axis = "trbl")

### Library size vs library complexity colored by percentate of mitocondrial genes

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 10, warn=-1,verbose = FALSE)
list_plots <- lapply(seq_along(list_objects), function(i) {
  id <- names(list_objects[i])
  obj <- list_objects[[i]]
  plt <- ggplot(obj@meta.data, aes(x = nCount_RNA, y = nFeature_RNA, color = pct_mt)) +
  geom_point() +
  ggtitle(id) + 
  theme(plot.title = element_text(hjust = 0.5))
  
  return(plt)
})
  cowplot::plot_grid(plotlist = list_plots,
                   align = "hv",
                   axis = "trbl")

#### Fraction of ribosomal expression

In [ ]:
list_plots <- lapply(seq_along(list_objects), function(i) {
  id <- names(list_objects[i])
  obj <- list_objects[[i]]
  plt <- obj@meta.data %>%
  plot_histogram_qc(x = "percent.ribo", x_lab = "% Ribosomal Expression") +
  geom_vline(xintercept = 60, linetype = "dashed", color = "black") +
  scale_x_continuous(limits = c(0, 100)) +
  ggtitle(id) + 
  theme(plot.title = element_text(hjust = 0.5))
    
  return(plt)
})
cowplot::plot_grid(plotlist = list_plots,
                   align = "hv",
                   axis = "trbl")

### Final application of filters

In [ ]:
filtering_QC <- function(seurat_obj, id) {
  
  df <- filters %>% dplyr::filter(sample == id)
  print(df)
  df$max_lib_size = as.numeric(df$max_lib_size)
  df$min_lib_size = as.numeric(df$min_lib_size)
  df$min_n_genes = as.numeric(df$min_n_genes)
  df$max_n_genes = as.numeric(df$max_n_genes)
  
  seurat_obj <- subset(
    x = seurat_obj, 
    pct_mt < max_pct_mt &
    nCount_RNA > df$min_lib_size &
    nCount_RNA < df$max_lib_size &
    nFeature_RNA < df$max_n_genes &
    nFeature_RNA > df$min_n_genes)
   
  return(seurat_obj)
}
list_objects <- lapply(seq_along(list_objects), function(i) {
  id <- names(list_objects[i])
  obj <- list_objects[[i]]
  obj <- filtering_QC(obj, id)
  return(obj)
})

In [ ]:
list_objects <- setNames(list_objects, list_names)

## Save clean objects

In [ ]:
saveRDS(list_objects, here::here(glue::glue("{qc}/{robj_dir}/clean_list_objects.rds")))
print(paste("Number of total filtered cells:", sum(melt(lapply(list_objects, ncol))$value)))

##  Session info

In [ ]:
sessionInfo()